<a href="https://colab.research.google.com/github/banno-0720/Deep-Learning-Projects/blob/main/Belief_State_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Belief State Transformer Paper Replication

Based on Research paper "THE BELIEF STATE TRANSFORMER
" which was published by Microsoft on February 20, 2025.

[Click on this](https://arxiv.org/pdf/2410.23506) for original research paper

And [Click on this](https://youtu.be/aqhbRtB2Fyg?si=ABz33R6ZfdWue-mi) for the video on the topic, for better understanding of Belief State Transformers

## Why replicate a machine learning research paper?

A machine learning research paper is often a presentation of months of work and experiments done by some of the best machine learning teams in the world condensed into a few pages of text.

And if these experiments lead to better results in an area related to the problem you're working on, it'd be nice to check them out.

Also, replicating the work of others is a fantastic way to practice your skills.

<img src="https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/images/08-george-hotz-quote.png" width=600 alt="george hotz quote saying to get better at being a machine learning engineer, download a paper, implement it and keep going until you have skills"/>

*George Hotz is founder of [comma.ai](https://comma.ai/), a self-driving car company and livestreams machine learning coding on [Twitch](https://www.twitch.tv/georgehotz) and those videos get posted in full to [YouTube](https://www.youtube.com/c/georgehotzarchive). I pulled this quote from one of his livestreams. The "٭" is to note that machine learning engineering often involves the extra step(s) of preprocessing data and making your models available for others to use (deployment).*

 ## What is a Belief State?

For any probability distribution over a set of sequences $P(x_{1:T})$, and for any partial sequence $s = x_{1:t}$, we define a vector $v_s$ to be a **belief state** for $s$ if there exists a randomized function $g$ such that

$$
g(v_s) \sim P(x_{t+1:T} \mid x_{1:t}).
$$

In other words, sampling $g(v_s)$ yields a sample from the conditional distribution $P(x_{t+1:T} \mid x_{1:t})$.


By definition, a belief state captures all available information relevant for predicting the future tokens.
Once the belief state is learned, there is no additional useful information to be gained—everything
necessary for future predictions is already encoded within it.



## Why use Belief State Transformer?

In the research paper, we see that they check its performance with
1. Star graph problem, in which it comes out to be the clear winner and with much higher accuracy than next-token, data-augmentation, FIM and teacherless models
2. TinyStories Dataset, it divides the text into three parts: prefix, suffix, and missing middle part and make the model predict the middle missing part. It gains very high accuracy

Due to its self evaluation feature, it does need high computational power but  it has almost perfect accuracy.

# 0. Getting Setup

In [ ]:
import os
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [ ]:
# Set up device agnostic code
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


# 1. Get Data

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

In [ ]:
# Load the dataset
dataset = load_dataset("mintujupally/ROCStories")
print(dataset)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Repo card metadata block was not found. Setting CardData to empty.


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 78528
    })
    test: Dataset({
        features: ['text'],
        num_rows: 19633
    })
})


# 2. Build a reduceded global vocabulary

Since we dont have enough computation resources and the vocab size of the datasets is too big to capture it all with the current computation resources which keeps causing out of memory errors.

We will reduce the vocab size to a max length of 10000 for the current model(thats the highest i could choose after all experimentation)

In [ ]:
from collections import Counter

In [ ]:
def build_vocab(hf_dataset, split="train", max_vocab_size=10000):
    counter = Counter()
    key = "text"
    for example in hf_dataset[split]:
        # The 'text' field is a list of sentences; join them into a single text.
        if isinstance(example[key], list):
            text = " ".join(example[key])
        else:
            text = example[key]
        tokens = text.split()  # Simple tokenization; for production, use a robust tokenizer.
        counter.update(tokens)
    # Add special tokens needed for our task.
    special_tokens = ["[PAD]", "[UNK]", "[PREFIX]", "[MISSING]", "[SUFFIX]"]
    most_common = counter.most_common(max_vocab_size - len(special_tokens))
    vocab = {token: idx for idx, token in enumerate(special_tokens)}
    for token, freq in most_common:
        if token not in vocab:
            vocab[token] = len(vocab)
    return vocab

In [ ]:
global_vocab = build_vocab(dataset, split="train", max_vocab_size=10000)
print("Global vocabulary size:", len(global_vocab))

Global vocabulary size: 10000


In [ ]:
# Use this reduced vocabulary size in the rest of the model.
vocab_size = len(global_vocab)

# 3. Creating Datasets and Dataloaders

In [ ]:
def transform_story(text, min_prefix_tokens=5, min_suffix_tokens=5):
    """
    Transforms a story into prefix, missing middle, and suffix parts.
    """
    tokens = text.split()  # Simple tokenization
    n = len(tokens)
    if n < (min_prefix_tokens + min_suffix_tokens + 1):
        return None
    prefix_end = random.randint(min_prefix_tokens, n - min_suffix_tokens - 1)
    missing_end = random.randint(prefix_end + 1, n - min_suffix_tokens)
    prefix = " ".join(tokens[:prefix_end])
    missing = " ".join(tokens[prefix_end:missing_end])
    suffix = " ".join(tokens[missing_end:])
    return prefix, missing, suffix

In [ ]:
class ROCStoriesDataset(Dataset):
    def __init__(self, hf_dataset, split="train", vocab=None, max_length=128):
        self.data = hf_dataset[split]
        self.vocab = vocab  # Use the global reduced vocab for tokenization.
        if "[UNK]" not in self.vocab:
            self.vocab["[UNK]"] = len(self.vocab)
        self.max_length = max_length  # Limit sequence length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Use the "text" key. If it's a list, join it into a single string.
        story_field = self.data[idx]["text"]
        if isinstance(story_field, list):
            text = " ".join(story_field)
        else:
            text = story_field

        transformed = transform_story(text)
        if transformed is None:
            transformed = (text, "", "")
        prefix, missing, suffix = transformed

        # Concatenate parts with special tokens.
        full_input = f"[PREFIX] {prefix} [MISSING] {missing} [SUFFIX] {suffix}"
        tokens = full_input.split()
        # Truncate to max_length tokens.
        tokens = tokens[:self.max_length]
        indices = [self.vocab.get(token, self.vocab["[UNK]"]) for token in tokens]
        x = torch.tensor(indices, dtype=torch.long)
        return x

In [ ]:
# Create datasets and dataloaders using ROCStories.
train_dataset = ROCStoriesDataset(dataset, split="train", vocab=global_vocab, max_length=128)
test_dataset = ROCStoriesDataset(dataset, split="test", vocab=global_vocab, max_length=128)

The batch size is also supposed to be 1. I have tried taking 2 and it caused out of memory error

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True,
                          collate_fn=lambda x: nn.utils.rnn.pad_sequence(x, batch_first=True, padding_value=0))
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False,
                        collate_fn=lambda x: nn.utils.rnn.pad_sequence(x, batch_first=True, padding_value=0))

In [ ]:
print("Train dataset size:", len(train_dataset))
print("Validation dataset size:", len(test_dataset))

Train dataset size: 78528
Validation dataset size: 19633


# 4. Belief State Transformer Overview

The Belief State Transformer is designed to capture a compact representation (or *belief state*) of a partial sequence. Given a sequence $ x_{1:T} $ and a partial sequence $ s = x_{1:t} $, the model learns a vector $ v_s $ such that a randomized function $ g(v_s) $ can sample from the conditional distribution $ P(x_{t+1:T} \mid x_{1:t}) $.

The architecture generally consists of:
- **Forward Encoder:** Processes the prefix $ x_{1:t} $ to produce forward states.
- **Backward Encoder:** Processes the sequence in reverse to capture future context.
- **Text Head:** Combines the forward and backward states to predict tokens in the missing (middle) section.
- **Efficient Prefix-Suffix Loss Computation:** Computes loss over all valid prefix-suffix pairs using a specialized loss function that handles multiple pairs efficiently.

This design allows the model to perform the "fill-in-the-middle" task effectively.


# 5. Equations

For any probability distribution over a set of sequences $ P(x_{1:T}) $, and for any partial sequence $ s = x_{1:t} $, we define a vector $ v_s $ to be a **belief state** for $ s $ if there exists a randomized function $ g $ such that:

$$
g(v_s) \sim P(x_{t+1:T} \mid x_{1:t}).
$$

The efficient computation of the prefix-suffix loss is given by:

1. Compute the forward state $ F $ from the prefix and the backward state $ B $ from the reversed sequence.
2. For each valid prefix-suffix pair, extract the corresponding states.
3. Concatenate the states and pass through the text head to obtain logits.
4. Reshape and compute the cross-entropy loss over all pairs.

The pseudocode for the loss function is provided in the research paper.


# 6. Creating the Transformer Encoder

This shows how generally a transformer encoder works

In [ ]:
class SimpleTransformerEncoder(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, hidden_dim, num_layers, dropout=0.1):
        super(SimpleTransformerEncoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, dim_feedforward=hidden_dim, dropout=dropout, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # x: (batch, seq_len)
        embedded = self.embedding(x)  # (batch, seq_len, embed_dim)
        # Transformer expects (seq_len, batch, embed_dim)
        embedded = embedded.transpose(0, 1)
        encoded = self.transformer_encoder(embedded)  # (seq_len, batch, embed_dim)
        encoded = encoded.transpose(0, 1)
        return self.dropout(encoded)

In [ ]:
transformer_encoder = SimpleTransformerEncoder(vocab_size, embed_dim=512, num_heads=8, hidden_dim=2048, num_layers=4).to(device)

# 7. Putting It All Together: Belief State Transformer

In [ ]:
class BeliefStateTransformer(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super(BeliefStateTransformer, self).__init__()
        # Forward encoder for the prefix
        self.enc_F = nn.Sequential(
            nn.Embedding(vocab_size, embed_dim),
            nn.Linear(embed_dim, embed_dim)
        )
        # Backward encoder for the suffix (applied on reversed sequence)
        self.enc_B = nn.Sequential(
            nn.Embedding(vocab_size, embed_dim),
            nn.Linear(embed_dim, embed_dim)
        )
        # Text head: combines forward and backward states to produce token logits.
        self.text_head = nn.Sequential(
            nn.Linear(embed_dim * 2, embed_dim),
            nn.LeakyReLU(),
            nn.Linear(embed_dim, vocab_size * 2)
        )

    def forward(self, x):
        # x: (batch, T)
        f = self.enc_F(x)  # (batch, T, embed_dim)
        b = self.enc_B(x)  # (batch, T, embed_dim)
        return f, b

In [ ]:
def belief_state_objective(all_f, all_b, text_head, x):
    """
    Efficient computation of the prefix-suffix loss.
    x: (batch, T) token indices.
    all_f: forward states from the encoder.
    all_b: backward states from the encoder.
    text_head: head network to produce logits.
    """
    bs, T = x.shape
    forward_state = all_f  # (batch, T, embed_dim)
    backward_state = all_b.flip(1)  # Reverse the sequence along T
    ft = torch.arange(T, dtype=torch.int32, device=x.device)
    bt = torch.arange(T, dtype=torch.int32, device=x.device)
    combinations = torch.cartesian_prod(ft, bt)  # All (i, j) pairs
    # Only consider pairs where j - i >= 2 and j < T
    combinations = combinations[(combinations[:, 1] - combinations[:, 0] >= 2)]
    fb_pairs = combinations.clone()
    fb_pairs = fb_pairs[combinations[:, 1] < T]
    f_idxs = fb_pairs[:, 0]
    b_idxs = fb_pairs[:, 1]
    nt_idxs = (combinations[:, 0] + 1)

    f_selected = forward_state[:, f_idxs]   # (batch, num_pairs, embed_dim)
    b_selected = backward_state[:, b_idxs]    # (batch, num_pairs, embed_dim)

    single_labels_f = x[:, nt_idxs].unsqueeze(2)  # (batch, num_pairs, 1)
    single_labels_b = x[:, b_idxs].unsqueeze(2)     # (batch, num_pairs, 1)
    single_labels = torch.cat((single_labels_f, single_labels_b), dim=2)  # (batch, num_pairs, 2)

    # Concatenate forward and backward states
    logits = text_head(torch.cat([f_selected, b_selected], dim=2))  # (batch, num_pairs, vocab_size*2)
    fb_numpairs = fb_pairs.shape[0]
    logits = logits.reshape((bs, fb_numpairs, 2, -1))  # (batch, num_pairs, 2, vocab_size)
    logits = logits.reshape((bs * fb_numpairs * 2, -1))
    single_labels = single_labels.reshape((bs * fb_numpairs * 2))
    loss = nn.CrossEntropyLoss()(logits, single_labels)
    return loss

In [ ]:
model = BeliefStateTransformer(vocab_size, embed_dim=512).to(device)

# 8. Setting up Training Code

In [ ]:
num_epochs = 3
learning_rate = 1e-3
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

model.train()
for epoch in range(num_epochs):
    epoch_loss = 0.0
    # Use tqdm to track progress over batches
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        batch = batch.to(device)

        # Forward pass: compute forward and backward states.
        f, b = model(batch)

        # Detach f and b for the objective computation, but retain gradients for backpropagation.
        _f = f.detach().requires_grad_()
        _b = b.detach().requires_grad_()

        optimizer.zero_grad()
        loss = belief_state_objective(_f, _b, model.text_head, batch)
        loss.backward()

        # Backpropagate the gradients to update encoders (simulate two-stage gradient update)
        f.backward(_f.grad)
        b.backward(_b.grad)

        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(train_loader):.4f}")

Epoch 1/3: 100%|██████████| 78528/78528 [35:04<00:00, 37.32it/s]


Epoch 1/3, Loss: 5.8891


Epoch 2/3: 100%|██████████| 78528/78528 [35:03<00:00, 37.34it/s]


Epoch 2/3, Loss: 5.8020


Epoch 3/3:  30%|███       | 23570/78528 [10:30<24:29, 37.41it/s]


KeyboardInterrupt: 

I had to stop the model training because i had some work to do and it already took 1 hour 30 mins to reach till here.

# 9. Make prediction on a custom input

In [ ]:
def predict_custom_input(model, input_text, vocab):
    """
    Given a custom input text, perform the fill-in-the-middle prediction,
    and map the predicted token index back to its corresponding word.
    """
    # Mimic the same tokenization as used in our dataset.
    tokens = input_text.split()
    indices = [vocab.get(token, vocab["[UNK]"]) for token in tokens]
    x = torch.tensor(indices, dtype=torch.long).unsqueeze(0).to(device)  # (1, seq_len)

    # Get forward and backward representations
    f, b = model(x)
    T = x.shape[1]
    if T < 4:
        print("Input too short for fill-in-the-middle prediction.")
        return
    prefix_idx = T // 3
    suffix_idx = T - T // 3

    # Extract forward state at prefix_idx and backward state at suffix_idx (from reversed sequence)
    f_state = f[:, prefix_idx:prefix_idx+1, :]  # (1, 1, embed_dim)
    b_state = b.flip(1)[:, suffix_idx:suffix_idx+1, :]  # (1, 1, embed_dim)

    # Combine and run through the text head to get logits
    combined = torch.cat([f_state, b_state], dim=2)
    logits = model.text_head(combined)
    logits = logits.reshape(-1, vocab_size)[0]
    predicted_token = torch.argmax(logits).item()

    # Map the predicted token index back to a word using the vocabulary.
    # Build a reverse vocabulary (index -> token).
    reverse_vocab = {idx: token for token, idx in vocab.items()}
    predicted_word = reverse_vocab.get(predicted_token, "[UNK]")

    print("Predicted token index:", predicted_token)
    print("Predicted token word:", predicted_word)

In [ ]:
# Example custom input
custom_story = "[PREFIX] Once upon a time, a brave knight [MISSING] [SUFFIX] embarked on a quest."
predict_custom_input(model, custom_story, global_vocab)

Predicted token index: 15
Predicted token word: he


This is just one word prediction lets try multiple words

In [ ]:
import torch.nn.functional as F

def top_k_sampling(logits, k=50, temperature=1.0):
    """
    Sample from the logits using top-k sampling with temperature.

    Args:
        logits (torch.Tensor): 1D tensor of shape (vocab_size,).
        k (int): Number of top candidates to sample from.
        temperature (float): Temperature parameter to control randomness.

    Returns:
        int: The sampled token index.
    """
    logits = logits / temperature
    topk_logits, topk_indices = torch.topk(logits, k)
    probabilities = F.softmax(topk_logits, dim=-1)
    sampled_index = torch.multinomial(probabilities, num_samples=1)
    return topk_indices[sampled_index].item()

def predict_missing_sequence(model, prefix_text, suffix_text, vocab, max_missing=20, k=50, temperature=1.0):
    """
    Given a prefix and suffix, iteratively generate a missing sequence using top-k sampling.

    Args:
        model: The BeliefStateTransformer model.
        prefix_text (str): The prefix part of the text.
        suffix_text (str): The suffix part of the text.
        vocab (dict): The vocabulary mapping tokens to indices.
        max_missing (int): Maximum number of tokens to generate.
        k (int): Number of top candidates to sample from.
        temperature (float): Temperature for sampling.

    Returns:
        str: The generated missing text.
    """
    # Build reverse vocab for mapping indices back to tokens.
    reverse_vocab = {idx: token for token, idx in vocab.items()}
    missing_tokens = []

    # Helper: convert text to tensor of token indices.
    def text_to_indices(text):
        tokens = text.split()
        indices = [vocab.get(token, vocab["[UNK]"]) for token in tokens]
        return torch.tensor(indices, dtype=torch.long).unsqueeze(0).to(device)

    prefix_tensor = text_to_indices(prefix_text)
    suffix_tensor = text_to_indices(suffix_text)

    for i in range(max_missing):
        # Get forward representation for the current prefix.
        f_prefix = model.enc_F(prefix_tensor)  # (1, L_prefix, embed_dim)
        f_state = f_prefix[:, -1, :]  # Last token representation

        # Get backward representation for the suffix.
        b_suffix = model.enc_B(suffix_tensor)  # (1, L_suffix, embed_dim)
        b_state = b_suffix.flip(1)[:, 0, :]  # First token of reversed sequence

        # Combine to form the belief state.
        combined_state = torch.cat([f_state, b_state], dim=1)  # (1, 2*embed_dim)
        logits = model.text_head(combined_state)  # (1, vocab_size*2)
        # Use only the first half of logits corresponding to our vocabulary.
        logits = logits[:, :vocab_size].squeeze(0)  # (vocab_size,)

        # Sample the next token using top-k sampling.
        next_index = top_k_sampling(logits, k=k, temperature=temperature)
        next_token = reverse_vocab.get(next_index, "[UNK]")

        # Stopping criteria: if a special token is predicted.
        if next_token in ["[SUFFIX]", "[PAD]"]:
            break

        missing_tokens.append(next_token)
        # Append the generated token to the prefix and update prefix_tensor.
        prefix_text = prefix_text + " " + next_token
        prefix_tensor = text_to_indices(prefix_text)

    missing_sequence = " ".join(missing_tokens)
    return missing_sequence

# Example usage:
prefix_example = "Once upon a time, a brave knight"
suffix_example = "embarked on a quest."
generated_missing = predict_missing_sequence(model, prefix_example, suffix_example, global_vocab, max_missing=20, k=50, temperature=1.0)
print("Generated missing sequence:", generated_missing)

Generated missing sequence: He would play football player. One day, she couldn't afford the most [UNK] in the most of fun it was


Okay well it doesnt work quite well because of lack of training and there is [unk] cuz of lack of vocab_size

But i think this is the best for now i can do with the resources that i have.